[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maplightrx/MapLight-TDC/blob/main/submission_gnn.ipynb)

__WARNING__: This notebook is not runnable on Colab because of [this issue](https://github.com/datamol-io/molfeat/issues/61). Please run it locally.

In [ ]:
!curl -o maplight_gnn.py https://raw.githubusercontent.com/maplightrx/MapLight-TDC/main/maplight_gnn.py

In [1]:
# installs for Colab
!pip install rdkit PyTDC tqdm catboost python-dotenv molfeat transformers torch_geometric dgl dgllife

  Using cached rdkit-2024.3.5-cp310-cp310-manylinux_2_28_x86_64.whl (33.1 MB)
  Using cached pytdc-1.1.0.tar.gz (145 kB)
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
  Using cached catboost-1.2.7-cp310-cp310-manylinux2014_x86_64.whl (98.7 MB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached molfeat-0.10.1-py3-none-any.whl (165 kB)
  Using cached transformers-4.45.2-py3-none-any.whl (9.9 MB)
  Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
  Using cached dgl-2.1.0-cp310-cp310-manylinux1_x86_64.whl (8.5 MB)
  Using cached dgllife-0.3.2-py3-none-any.whl (226 kB)
  Using cached pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.4 MB)
  Using cached numpy-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Using cached accelerate-0.33.0-py3-none-any.whl (315 kB)
  Using cached biopython-1.84-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Using cached ce


KeyboardInterrupt



In [ ]:
!conda run -n maplight python -m pip install scipy h5py
!conda run -n maplight python -m pip install ipykernel
!conda run -n maplight python -m python -m ipykernel install --user --name maplight
!conda run -n maplight python -m pip install pandas==1.*

In [ ]:
!conda install -n maplight -c dglteam dgl

In [1]:
from maplight_gnn import *
from tqdm import tqdm
import catboost as cb

from tdc.benchmark_group import admet_group

In [2]:
from tdc.utils import create_scaffold_split, create_fold
import pandas as pd
import rdkit.Chem as Chem
df_tox = pd.read_csv('data/tox21_hek293_combined_assays.csv')
tox_data = {}
frac = frac = [0.875, 0.125, 0.0]
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    canonical_smiles = Chem.MolToSmiles(mol, canonical=True)
    return canonical_smiles
df_tox['smiles'] = df_tox.smiles.map(canonicalize_smiles)
for column in df_tox.columns[4:]:
    dataset = df_tox[['smiles',column]].rename(columns={'smiles':"Drug", column:"Y"}).dropna()
    #train, test, _ = create_scaffold_split(dataset, 42,frac, 'DRUG')
    folds = create_fold(dataset, 42, frac)
    data = {
        'name': column,
        'train_val': folds['train'],
        'test': folds['valid']
    }
    tox_data[column] = data
tox_data

[22:26:48] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 16
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11 12 13 14 15
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 24 25 26 27 28 31 32 33 34
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 26 27 28 29 30 33 34 35 36
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 9 10
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18 19 20 21 22
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 7 16 17
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15 16 17 18 21
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 32 33 34 35 36
[22:26:48] Can't kekulize mol.  Unkekulized atoms: 2

{'tox21-ar-bla-agonist-p1': {'name': 'tox21-ar-bla-agonist-p1',
  'train_val':                                                    Drug    Y
  0     C[N+]1(C)[C@@H]2CC[C@H]1C[C@@H](OC(=O)C(CO)c1c...  0.0
  1     C[N+]1([O-])[C@@H]2CC[C@H]1C[C@@H](OC(=O)C(CO)...  0.0
  2                                         O=C(O)C1CCCN1  0.0
  3                    CN(C)CCCN1c2ccccc2C(C)(C)c2ccccc21  0.0
  4     CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@...  0.0
  ...                                                 ...  ...
  6920     CC(=O)CCC(=O)[O-].CC(=O)CCC(=O)[O-].O.O.[Ca+2]  0.0
  6921  O=C(Nc1ccc(S(=O)(=O)N2CCOCC2)cc1)c1cc(Cl)ccc1N...  0.0
  6922                                       NC(CS)C(=O)O  0.0
  6923  CC(=O)Oc1ccc2c(c1)CC[C@@H]1[C@@H]2CC[C@]2(C)[C...  1.0
  6924                                COC(=O)C(Cl)C(=O)OC  0.0
  
  [6925 rows x 2 columns],
  'test':                                                   Drug    Y
  0              CCOC(=O)c1ncn2c1CN(C)C(=O)c1c(I)cccc1-2  0.0
  1  

In [3]:
from sklearn.metrics import average_precision_score, roc_auc_score, mean_absolute_error, balanced_accuracy_score
from scipy.stats import spearmanr
#Quick function to replace the native TDC one for tox data
def evaluate_many(predictions_list):
    metric = []
    for x in predictions_list:
        for k,v in x.items():
            metric.append(average_precision_score(tox_data[k]['test'].Y, v))
    return {k:sum(metric)/float(len(metric))}
def evaluate_aupr(true,pred):
    return average_precision_score(true,pred)
def evaluate_auroc(true,pred):
    return roc_auc_score(true,pred)
def evaluate_ba(true,pred):
    return balanced_accuracy_score(true,pred)
def evaluate_mae(true,pred):
    return mean_absolute_error(true,pred)
def evaluate_spearman(true,pred):
    return spearmanr(true,pred)

In [5]:
import pandas as pd
import numpy as np
!pip install tabulate
from tabulate import tabulate

df = pd.DataFrame.from_dict(metrics)#, orient='index')
df.spearman = df['spearman'].apply(lambda x: x.statistic if not isinstance(x, float) else x)
# Generate HTML table string using tabulate
html_table = tabulate(df[['mae','spearman']].dropna(), headers='keys', tablefmt="github")#, tablefmt='html')
print(html_table)
html_table = tabulate(df[['aupr','auroc','balacc','optimal_th']].dropna(), headers='keys', tablefmt="github")#, tablefmt='html')
print(html_table)
df.to_csv('data/metrics.csv')

ModuleNotFoundError: No module named 'tabulate'

In [6]:
# Infer
from collections import defaultdict
#print(benchmark_config)
predictions = defaultdict(dict)
for k,v in df_inf_d.items():
    print(k)
    test = v
    seed = 5
    X_test = get_fingerprints(test['Drug'])
    for name,admet_benchmark in benchmark_config.items():
        task = admet_benchmark[0]
        print(name)
        params = {
                'random_strength': 2, 
                'random_seed': seed,
                'verbose': 0,
            }
        if task == 'regression':
            params['loss_function'] = 'MAE'            
            model = cb.CatBoostRegressor(**params)
            model.load_model(f"{name}")
            y_pred_test = model.predict(X_test)
            print(y_pred_test)
        elif task == 'binary':
            params['loss_function'] = 'Logloss'
            model = cb.CatBoostClassifier(**params)
            model.load_model(f"{name}")
            y_pred_test = model.predict_proba(X_test)[:, 1]
        predictions[k][name] = y_pred_test

NameError: name 'df_inf_d' is not defined

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False).fit(X)
ohe.transform()

NameError: name 'X' is not defined

In [8]:
import pandas as pd
df_d={}
for k in ['passed_phase_1','passed_phase_2','passed_phase_3']:
    df_d[k] = pd.read_csv(f'/efs-private/alladmet/NB_Pred_data_{k}.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/efs-private/alladmet/NB_Pred_data_passed_phase_1.csv'

In [ ]:
df_inf_d={}
for k in df_d:
    df_inf_d[k]=pd.DataFrame()
    df_inf_d[k]['Drug_ID'] = df_d[k].smiles.unique()
    df_inf_d[k]['Drug'] = df_d[k].smiles.unique()
    df_inf_d[k]['Y'] = 0

In [ ]:
X = defaultdict(lambda: pd.DataFrame())
for k,v in predictions.items():
    for name,arr in v.items():
        X[k][name] = arr

In [ ]:
X = {k:pd.concat([df_inf_d[k],v],axis=1).rename(columns={'Drug':'smiles'}) for k,v in X.items()}
for k,v in X.items():
    v.to_csv(f'predictions_{k}.csv')

In [ ]:
#df = df[['smiles','indication','final_status']].drop_duplicates()


In [ ]:
#X=X.rename(columns={'Drug':'smiles'}) #.drop(columns='Y')
df_d_ = {k:v.join(X[k].set_index('smiles'), on='smiles') for k,v in df_d.items()} #.drop(columns=['Drug_ID','Y'])
for k in df_d_:
    #df_d[k]['Y'] = (df_d[k].y == 'Failed').astype(int)
    #df_d_[k]
    df_d_[k]=df_d_[k][df_d_[k].y != "Not tested"]
    df_d_[k]['Y'] = df_d_[k]['y'].isin([False,'False']).astype(int)
    #df_d_[k].rename(columns={'y':'Y'})
#df

In [ ]:
df_d_

In [ ]:
#NOT NEEDED WITH CATBOOST!!! IT OHEs itself
from sklearn.preprocessing import OneHotEncoder
# data is a Pandas DataFrame
data={}
for k,v in df_d_.items():
    jobs_encoder = OneHotEncoder(sparse=False)
    jobs_encoder.fit(v['indication'].unique().reshape(-1, 1))
    transformed = jobs_encoder.transform(v['indication'].to_numpy().reshape(-1,1))
    #Create a Pandas DataFrame of the hot encoded column
    ohe_df = pd.DataFrame(transformed, columns=jobs_encoder.get_feature_names_out())
    #concat with original data
    print(v.shape)
    print(ohe_df.shape)
    data[k] = pd.concat([v.reset_index(), ohe_df], axis=1)#.drop(columns=['smiles', 'Unnamed: 0',k, 'indication','Drug_ID'])
    #data.to_csv('all_w_ohe.csv')

In [ ]:
#data['passed_phase_1'].columns

In [ ]:
X, y = data.drop(columns='Y').to_numpy(), data.Y.to_numpy()

In [ ]:
df_in = df_d_
drop = ['Unnamed: 0', 'Drug_ID','y']
#df_in = data
#drop = ['Unnamed: 0', 'Drug_ID','y','index','indication','smiles']
df_in = {k:v.sample(frac=1).reset_index(drop=True) for k,v in df_in.items()}
df_xy_d = {k: v.drop(columns=drop) for k,v in df_in.items()}
df_xx_d = {k: v.drop(columns='Y') for k,v in df_xy_d.items()}
df_yy_d = {k: v.Y for k,v in df_xy_d.items()}
#X_test_d = {k: get_fingerprints(v['smiles']) for k,v in df_xx_d.items()}

In [ ]:
df_xx_d = {k: pd.concat([v.reset_index(drop=True),pd.DataFrame(X_test_d[k])],axis=1).drop('smiles',axis=1) for k,v in df_xx_d.items()}
#df_xx_d={k:v.drop(columns='index') for k,v in df_xx_d.items()}

In [ ]:
df_xx_d

In [ ]:
c_weight = {k: len(v)/sum(v) for k,v in df_yy_d.items()}
c_weight

In [ ]:
df_xx_d = {k:v.drop(columns=v.columns[1:]) for k,v in df_xx_d.items()}

In [9]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt
for k in df_d_.keys():
    print(f"-----------------")
    print(k)
    print(f"-----------------")

    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    scores = defaultdict(list)
    for i, (train_index, test_index) in enumerate(kf.split(df_xx_d[k])):
        #print(f"Fold {i}:")
        #df_xx_d[k].columns = df_xx_d[k].columns.astype(str)
        X_train, X_test, y_train, y_test = (df_xx_d[k].iloc[train_index],
                                            df_xx_d[k].iloc[test_index],
                                            df_yy_d[k].iloc[train_index],
                                            df_yy_d[k].iloc[test_index])
        #X_train, X_test, y_train, y_test = train_test_split(df_xx_d[k], df_yy_d[k], test_size=0.1, random_state=r)
        #print(y_train.sum()/len(y_train))
        #gnb = GaussianNB()
        #s_weights = [c_weight[k] if x == 1 else 1.0 for x in y_train ]
        #gnb = LogisticRegression()
        #y_pred = gnb.fit(X_train, y_train, sample_weight=s_weights).predict(X_test)
        gnb = cb.CatBoostClassifier(loss_function = 'Logloss', 
                                    simple_ctr='Counter',
                                    combinations_ctr='Counter',
                                    cat_features=['indication'],
                                    class_weights=[1.0,c_weight[k]],
                                    verbose=0)
        y_pred = gnb.fit(X_train, y_train).predict(X_test)
        """
        print('---------')
        print("Number of mislabeled points out of a total %d points : %d"
              % (X_test.shape[0], (y_test != y_pred).sum()))
        print(f"{gnb.score(X_test,y_test)=}")
        print(f"{average_precision_score(y_test, y_pred)=}")
        print(f"{roc_auc_score(y_test,y_pred)=}")
        print(f"{f1_score(y_test,y_pred)=}")
        """
        scores['loss'].append(gnb.score(X_test,y_test))
        scores['ap'].append(average_precision_score(y_test, y_pred))
        scores['roc'].append(roc_auc_score(y_test,y_pred))
        scores['f1'].append(f1_score(y_test,y_pred))
        # = plt.subplots()
        #ConfusionMatrixDisplay.from_estimator(gnb, X_test, y_test)
        #plt.show()
    for k,v in scores.items():
        print(f"{k}:{sum(v)/len(v)}")

NameError: name 'df_d_' is not defined

In [ ]:
df_xx_d = {k:v.reset_index(drop=True) for k,v in df_xx_d.items()}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

parameters = {'depth' : [6,7,8,12],
              'learning_rate' : [0.1,.15,.2],
              'iterations'    : [80,90,100,120,140],
              }
grid = {}
for k,v in df_xx_d.items():
    model = cb.CatBoostClassifier(loss_function = 'Logloss', 
                            simple_ctr='Counter',
                            combinations_ctr='Counter',
                            cat_features=['indication'],
                            class_weights=[1.0,c_weight[k]],
                            verbose=0)
    grid[k] = GridSearchCV(estimator=model, 
                        param_grid = parameters, 
                        cv = 3, 
                        n_jobs=-1, 
                        verbose=3,
                       #scoring='f1'
                        scoring=['f1','balanced_accuracy','average_precision','roc_auc'],
                        refit=False
                       )
    grid[k].fit(df_xx_d[k],df_yy_d[k])

[CV 3/3] END depth=12, iterations=100, learning_rate=0.1; average_precision: (test=0.249) balanced_accuracy: (test=0.633) f1: (test=0.272) roc_auc: (test=0.752) total time= 1.1min
[CV 3/3] END depth=12, iterations=120, learning_rate=0.2; average_precision: (test=0.242) balanced_accuracy: (test=0.621) f1: (test=0.279) roc_auc: (test=0.725) total time= 1.3min
[CV 1/3] END depth=6, iterations=80, learning_rate=0.2; average_precision: (test=0.517) balanced_accuracy: (test=0.741) f1: (test=0.517) roc_auc: (test=0.816) total time=  11.8s
[CV 3/3] END depth=6, iterations=120, learning_rate=0.1; average_precision: (test=0.502) balanced_accuracy: (test=0.731) f1: (test=0.488) roc_auc: (test=0.816) total time=  14.2s
[CV 1/3] END depth=7, iterations=80, learning_rate=0.1; average_precision: (test=0.506) balanced_accuracy: (test=0.758) f1: (test=0.517) roc_auc: (test=0.812) total time=  13.6s
[CV 2/3] END depth=7, iterations=80, learning_rate=0.2; average_precision: (test=0.523) balanced_accuracy

[CV 1/3] END depth=12, iterations=100, learning_rate=0.15; average_precision: (test=0.197) balanced_accuracy: (test=0.610) f1: (test=0.248) roc_auc: (test=0.767) total time= 1.2min
[CV 2/3] END depth=12, iterations=140, learning_rate=0.1; average_precision: (test=0.196) balanced_accuracy: (test=0.622) f1: (test=0.285) roc_auc: (test=0.749) total time= 1.4min
[CV 1/3] END depth=6, iterations=90, learning_rate=0.1; average_precision: (test=0.492) balanced_accuracy: (test=0.742) f1: (test=0.489) roc_auc: (test=0.812) total time=  11.4s
[CV 1/3] END depth=6, iterations=120, learning_rate=0.1; average_precision: (test=0.505) balanced_accuracy: (test=0.746) f1: (test=0.506) roc_auc: (test=0.814) total time=  15.2s
[CV 2/3] END depth=7, iterations=80, learning_rate=0.1; average_precision: (test=0.555) balanced_accuracy: (test=0.754) f1: (test=0.511) roc_auc: (test=0.836) total time=  14.5s
[CV 1/3] END depth=7, iterations=90, learning_rate=0.2; average_precision: (test=0.521) balanced_accurac

[CV 2/3] END depth=12, iterations=100, learning_rate=0.1; average_precision: (test=0.185) balanced_accuracy: (test=0.638) f1: (test=0.298) roc_auc: (test=0.748) total time= 1.2min
[CV 1/3] END depth=12, iterations=140, learning_rate=0.1; average_precision: (test=0.212) balanced_accuracy: (test=0.621) f1: (test=0.264) roc_auc: (test=0.771) total time= 1.4min
[CV 2/3] END depth=6, iterations=80, learning_rate=0.2; average_precision: (test=0.543) balanced_accuracy: (test=0.763) f1: (test=0.537) roc_auc: (test=0.836) total time=  11.3s
[CV 2/3] END depth=6, iterations=100, learning_rate=0.15; average_precision: (test=0.526) balanced_accuracy: (test=0.756) f1: (test=0.524) roc_auc: (test=0.831) total time=  10.7s
[CV 2/3] END depth=6, iterations=120, learning_rate=0.2; average_precision: (test=0.544) balanced_accuracy: (test=0.742) f1: (test=0.525) roc_auc: (test=0.837) total time=  14.9s
[CV 3/3] END depth=7, iterations=80, learning_rate=0.15; average_precision: (test=0.519) balanced_accur

[CV 1/3] END depth=12, iterations=120, learning_rate=0.15; average_precision: (test=0.196) balanced_accuracy: (test=0.602) f1: (test=0.236) roc_auc: (test=0.769) total time= 1.5min
[CV 2/3] END depth=6, iterations=80, learning_rate=0.1; average_precision: (test=0.507) balanced_accuracy: (test=0.751) f1: (test=0.495) roc_auc: (test=0.824) total time=  11.1s
[CV 1/3] END depth=6, iterations=100, learning_rate=0.15; average_precision: (test=0.524) balanced_accuracy: (test=0.758) f1: (test=0.538) roc_auc: (test=0.822) total time=  13.5s
[CV 2/3] END depth=6, iterations=140, learning_rate=0.1; average_precision: (test=0.537) balanced_accuracy: (test=0.742) f1: (test=0.504) roc_auc: (test=0.827) total time=  16.3s
[CV 1/3] END depth=7, iterations=90, learning_rate=0.15; average_precision: (test=0.505) balanced_accuracy: (test=0.733) f1: (test=0.506) roc_auc: (test=0.813) total time=  14.0s
[CV 1/3] END depth=7, iterations=120, learning_rate=0.15; average_precision: (test=0.524) balanced_accu

[CV 2/3] END depth=12, iterations=120, learning_rate=0.15; average_precision: (test=0.195) balanced_accuracy: (test=0.623) f1: (test=0.288) roc_auc: (test=0.739) total time= 1.5min
[CV 3/3] END depth=6, iterations=80, learning_rate=0.1; average_precision: (test=0.480) balanced_accuracy: (test=0.726) f1: (test=0.473) roc_auc: (test=0.807) total time=  11.5s
[CV 2/3] END depth=6, iterations=100, learning_rate=0.2; average_precision: (test=0.550) balanced_accuracy: (test=0.758) f1: (test=0.539) roc_auc: (test=0.837) total time=  13.4s
[CV 2/3] END depth=6, iterations=140, learning_rate=0.15; average_precision: (test=0.518) balanced_accuracy: (test=0.712) f1: (test=0.481) roc_auc: (test=0.829) total time=  16.6s
[CV 3/3] END depth=7, iterations=90, learning_rate=0.2; average_precision: (test=0.501) balanced_accuracy: (test=0.709) f1: (test=0.489) roc_auc: (test=0.814) total time=  14.4s
[CV 3/3] END depth=7, iterations=120, learning_rate=0.15; average_precision: (test=0.514) balanced_accur

[CV 3/3] END depth=12, iterations=120, learning_rate=0.1; average_precision: (test=0.249) balanced_accuracy: (test=0.630) f1: (test=0.284) roc_auc: (test=0.750) total time= 1.5min
[CV 1/3] END depth=6, iterations=80, learning_rate=0.1; average_precision: (test=0.487) balanced_accuracy: (test=0.734) f1: (test=0.478) roc_auc: (test=0.809) total time=  11.1s
[CV 3/3] END depth=6, iterations=100, learning_rate=0.1; average_precision: (test=0.493) balanced_accuracy: (test=0.731) f1: (test=0.483) roc_auc: (test=0.812) total time=  13.2s
[CV 1/3] END depth=6, iterations=140, learning_rate=0.1; average_precision: (test=0.509) balanced_accuracy: (test=0.744) f1: (test=0.515) roc_auc: (test=0.815) total time=  15.5s
[CV 3/3] END depth=7, iterations=80, learning_rate=0.2; average_precision: (test=0.500) balanced_accuracy: (test=0.720) f1: (test=0.497) roc_auc: (test=0.812) total time=  10.8s
[CV 3/3] END depth=7, iterations=100, learning_rate=0.15; average_precision: (test=0.517) balanced_accurac

In [ ]:
#!conda run -n maplight pip install tabulate
from tabulate import tabulate
from collections import defaultdict
data = defaultdict(list)
phases = ['passed_phase_1','passed_phase_2','passed_phase_3']
mets = ['mean_test_f1','mean_test_average_precision','mean_test_roc_auc','mean_test_balanced_accuracy']
data['metric']=mets
for j in phases:
    for k in mets:
        data[j].append(pd.DataFrame(grid[j].cv_results_).sort_values(k,ascending=False)[k].head(1).values)
print(data)
tabulate(data,tablefmt='html', headers = data.keys())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=r)
#X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.1, random_state=r)
model = GaussianNB()
#s_weights = [c_weight if x == 1 else 1.0 for x in y_train ]


parameters = {
    'var_smoothing': np.logspace(1,-3, num=100)
}

grid = GridSearchCV(estimator=model, 
                    param_grid = parameters, 
                    cv = 5, 
                    n_jobs=-1, 
                    verbose=3,
                   scoring='f1')
grid.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from fastFM import sgd
fm = sgd.FMClassification(n_iter=1000, init_stdev=0.1, l2_reg_w=0,
                          l2_reg_V=0, rank=2, step_size=0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=r)
#X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.1, random_state=r)
#model = GaussianNB()
#s_weights = [c_weight if x == 1 else 1.0 for x in y_train ]
y_labels = np.ones_like(y_train)
y_labels[y_train == 0 ] = -1
y_train=y_labels
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)

In [ ]:
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=5)
clf = GaussianNB()
#clf = LogisticRegression()
print(f"roc: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='roc_auc').mean()}")
print(f"f1: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='f1').mean()}")
print(f"ap: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='average_precision').mean()}")


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=5)
#clf = GaussianNB()
clf = LogisticRegression()
print(f"roc: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='roc_auc').mean()}")
print(f"f1: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='f1').mean()}")
print(f"ap: {cross_val_score(clf, X, y, cv=k_fold, n_jobs=1, scoring='average_precision').mean()}")

In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

imps_test = permutation_importance(gnb, X_test, y_test, n_repeats=30)
print(imps_test.importances_mean)
imps_train = permutation_importance(gnb, X_train, y_train, n_repeats=30)
print(imps_train.importances_mean)

In [ ]:
def plot_permutation_importance(clf, X, y, ax):
    result = permutation_importance(clf, X, y, n_repeats=100, random_state=42, scoring='f1')#, n_jobs=2)
    perm_sorted_idx = result.importances_mean.argsort()

    ax.boxplot(
        result.importances[perm_sorted_idx].T,
        vert=False,
        labels=X.columns[perm_sorted_idx],
    )
    ax.axvline(x=0, color="k", linestyle="--")
    return ax

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (5,15))
_ = plot_permutation_importance(gnb, X_test, y_test, ax)
#print(print(f"Baseline accuracy on test data: {gnb.score(X_train, y_train):.2}"))

In [ ]:
fig, ax = plt.subplots(figsize = (5, 15))
_ = plot_permutation_importance(gnb, X_train, y_train, ax)
ax.set_xlabel(r'$\longleftarrow$ Importance')